# 🎯 CodeBERT Clustering Optimization

**Goal:** Reach 0.4+ silhouette score using CodeBERT embeddings

**Previous Results:**
- CodeBERT K=5: **0.2517** (7x better than baseline!)
- CodeBERT K=10: 0.1569
- Best alternative (BGE K=15): 0.1149

**Why CodeBERT works:** Natural split between Code MMLU (coding questions) and MMLU (academic questions)

**Experiments:**
1. Test lower K values (K=2,3,4) - might match natural data structure
2. Different clustering algorithms (HDBSCAN, GMM, Spectral)
3. Hierarchical clustering (2-level approach)
4. Detailed visualization of clusters

---
**📝 Designed for Google Colab** - GPU recommended for faster processing!

## 0. 🔧 Colab Setup

In [ ]:
# Install required packages
!pip install -q sentence-transformers datasets umap-learn scikit-learn matplotlib seaborn pandas numpy hdbscan

print("✅ All packages installed!")

In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    device = 'cuda'
    print(f"🎮 GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = 'cpu'
    print("💻 Running on CPU")
    print("   💡 To enable GPU: Runtime → Change runtime type → GPU")

## 1. 📦 Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Dimensionality reduction
from sklearn.manifold import TSNE
import umap

# Clustering algorithms
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
import hdbscan

# Metrics
from sklearn.metrics import silhouette_score, silhouette_samples, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import normalize

# Embeddings
from sentence_transformers import SentenceTransformer

# Data loading
from datasets import load_dataset

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Set seeds
np.random.seed(42)

print("✅ Imports complete!")

## 2. 📥 Load Data (Same as Before)

In [ ]:
def load_training_data(max_samples=3500):
    """
    Load MMLU + Code MMLU data:
    - 16 MMLU subjects (academic questions)
    - 9 Code MMLU subsets (coding questions)
    """
    questions = []
    
    # MMLU subjects (academic)
    mmlu_subsets = [
        "college_computer_science",
        "high_school_computer_science",
        "college_mathematics",
        "high_school_mathematics",
        "formal_logic",
        "machine_learning",
        "college_physics",
        "high_school_physics",
        "computer_security",
        "electrical_engineering",
        "abstract_algebra",
        "astronomy",
        "elementary_mathematics",
        "college_chemistry",
        "high_school_chemistry",
        "conceptual_physics",
    ]
    
    # Code MMLU subsets (coding)
    code_mmlu_subsets = [
        "api_frameworks",
        "code_completion",
        "code_repair",
        "dbms_sql",
        "execution_prediction",
        "fill_in_the_middle",
        "others",
        "programming_syntax",
        "software_principles",
    ]
    
    total_sources = len(mmlu_subsets) + len(code_mmlu_subsets)
    questions_per_source = max_samples // total_sources
    
    print("="*60)
    print("LOADING DATA (MMLU + Code MMLU)")
    print("="*60)
    print(f"Target: ~{max_samples} questions total")
    print(f"Questions per source: ~{questions_per_source}")
    print()
    
    # 1. Load MMLU subjects
    print(f"1. Loading {len(mmlu_subsets)} MMLU subjects...")
    for subset in mmlu_subsets:
        try:
            dataset = load_dataset("cais/mmlu", subset, split="test", trust_remote_code=True)
            count = 0
            
            for idx, item in enumerate(dataset):
                if count >= questions_per_source:
                    break
                    
                question_text = item.get("question", "")
                choices = item.get("choices", [])
                
                if question_text and len(choices) == 4:
                    questions.append({
                        "question_id": f"mmlu_{subset}_{idx}",
                        "question": question_text,
                        "category": f"mmlu_{subset}",
                        "data_type": "academic",
                    })
                    count += 1
        except Exception as e:
            print(f"   ✗ {subset}: {e}")
    
    print(f"   ✓ Loaded {sum(1 for q in questions if q['data_type'] == 'academic')} academic questions")
    
    # 2. Load Code MMLU subsets
    print(f"\n2. Loading {len(code_mmlu_subsets)} Code MMLU subsets...")
    for subset in code_mmlu_subsets:
        try:
            dataset = load_dataset("Fsoft-AIC/CodeMMLU", subset, split="test", trust_remote_code=True)
            count = 0
            
            for idx, item in enumerate(dataset):
                if count >= questions_per_source:
                    break
                    
                question_text = item.get("question", "")
                
                if question_text:
                    questions.append({
                        "question_id": f"code_mmlu_{subset}_{idx}",
                        "question": question_text,
                        "category": f"code_mmlu_{subset}",
                        "data_type": "code",
                    })
                    count += 1
        except Exception as e:
            print(f"   ✗ {subset}: {e}")
    
    print(f"   ✓ Loaded {sum(1 for q in questions if q['data_type'] == 'code')} code questions")
    
    print(f"\n{'='*60}")
    print(f"✅ Total: {len(questions)} questions")
    print(f"   Academic: {sum(1 for q in questions if q['data_type'] == 'academic')}")
    print(f"   Code: {sum(1 for q in questions if q['data_type'] == 'code')}")
    print(f"   Categories: {len(set(q['category'] for q in questions))}")
    print(f"{'='*60}")
    
    return questions

# Load data
questions = load_training_data(max_samples=3500)

## 3. 🧠 Extract CodeBERT Embeddings

In [ ]:
print("="*60)
print("LOADING CODEBERT")
print("="*60)

# Load CodeBERT model
model = SentenceTransformer("microsoft/codebert-base", device=device)
print(f"✅ Model loaded on {device}")

# Extract texts
texts = [q["question"] for q in questions]
data_types = [q["data_type"] for q in questions]
categories = [q["category"] for q in questions]

print(f"\nEncoding {len(texts)} questions...")
embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=False,
    device=device
)

print(f"\n✅ Embeddings shape: {embeddings.shape}")
print(f"   Mean: {embeddings.mean():.4f}, Std: {embeddings.std():.4f}")

# Normalize
embeddings_norm = normalize(embeddings, norm='l2')
print(f"✅ Normalized embeddings ready")

# Free GPU memory
del model
if device == 'cuda':
    torch.cuda.empty_cache()

## 4. 🎨 Visualize Embeddings (Code vs Academic Split)

In [ ]:
# Create binary labels for visualization
binary_labels = [0 if dt == "academic" else 1 for dt in data_types]

print("Running UMAP for visualization...")
reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1, metric='cosine')
embeddings_2d = reducer.fit_transform(embeddings_norm)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Binary (Code vs Academic)
colors_binary = ['#FF6B6B' if label == 0 else '#4ECDC4' for label in binary_labels]
ax1.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors_binary, alpha=0.6, s=20, edgecolors='black', linewidth=0.3)
ax1.set_title('CodeBERT: Code vs Academic Split', fontsize=14, fontweight='bold')
ax1.set_xlabel('UMAP 1')
ax1.set_ylabel('UMAP 2')
ax1.legend(handles=[
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#FF6B6B', markersize=10, label='Academic (MMLU)'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#4ECDC4', markersize=10, label='Code (Code MMLU)')
])
ax1.grid(alpha=0.3)

# Plot 2: All 25 categories
unique_categories = sorted(set(categories))
category_colors = {cat: i for i, cat in enumerate(unique_categories)}
colors_cat = [category_colors[cat] for cat in categories]
scatter = ax2.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors_cat, cmap='tab20', alpha=0.6, s=20, edgecolors='black', linewidth=0.3)
ax2.set_title('CodeBERT: All 25 Categories', fontsize=14, fontweight='bold')
ax2.set_xlabel('UMAP 1')
ax2.set_ylabel('UMAP 2')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualization complete")
print(f"\n💡 Observation: Clear separation between Code (cyan) and Academic (red) clusters!")

## 5. 🔬 Experiment 1: Lower K Values (K=2,3,4)

Hypothesis: K=2 or K=3 might match natural data structure better

In [ ]:
# Test lower K values
k_values = [2, 3, 4, 5, 6, 8, 10, 15, 20, 25, 30]
results = []

print("="*60)
print("TESTING K-MEANS WITH DIFFERENT K")
print("="*60)

for k in k_values:
    print(f"K={k:2d}: ", end='')
    
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(embeddings_norm)
    
    # Multiple metrics
    silhouette = silhouette_score(embeddings_norm, labels, metric='cosine')
    calinski = calinski_harabasz_score(embeddings_norm, labels)
    davies = davies_bouldin_score(embeddings_norm, labels)
    
    results.append({
        'k': k,
        'silhouette': silhouette,
        'calinski_harabasz': calinski,
        'davies_bouldin': davies,
        'inertia': kmeans.inertia_
    })
    
    status = "✅ TARGET!" if silhouette >= 0.4 else ""
    print(f"Silhouette={silhouette:.4f} {status}")

results_df = pd.DataFrame(results)
print(f"\n✅ Testing complete!")

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Silhouette
axes[0,0].plot(results_df['k'], results_df['silhouette'], marker='o', linewidth=2, markersize=8, color='steelblue')
axes[0,0].axhline(y=0.4, color='green', linestyle='--', linewidth=2, label='🎯 Target: 0.4')
axes[0,0].axhline(y=0.0353, color='red', linestyle='--', linewidth=2, alpha=0.5, label='📊 Baseline: 0.0353')
axes[0,0].set_xlabel('K')
axes[0,0].set_ylabel('Silhouette Score')
axes[0,0].set_title('Silhouette Score vs K', fontweight='bold')
axes[0,0].legend()
axes[0,0].grid(alpha=0.3)

# Calinski-Harabasz (higher is better)
axes[0,1].plot(results_df['k'], results_df['calinski_harabasz'], marker='s', linewidth=2, markersize=8, color='coral')
axes[0,1].set_xlabel('K')
axes[0,1].set_ylabel('Calinski-Harabasz Index')
axes[0,1].set_title('Calinski-Harabasz Index vs K (↑ better)', fontweight='bold')
axes[0,1].grid(alpha=0.3)

# Davies-Bouldin (lower is better)
axes[1,0].plot(results_df['k'], results_df['davies_bouldin'], marker='^', linewidth=2, markersize=8, color='purple')
axes[1,0].set_xlabel('K')
axes[1,0].set_ylabel('Davies-Bouldin Index')
axes[1,0].set_title('Davies-Bouldin Index vs K (↓ better)', fontweight='bold')
axes[1,0].grid(alpha=0.3)

# Inertia
axes[1,1].plot(results_df['k'], results_df['inertia'], marker='D', linewidth=2, markersize=8, color='teal')
axes[1,1].set_xlabel('K')
axes[1,1].set_ylabel('Inertia')
axes[1,1].set_title('Inertia vs K (Elbow Method)', fontweight='bold')
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Print best
best = results_df.loc[results_df['silhouette'].idxmax()]
print("\n" + "="*60)
print("BEST K-MEANS CONFIGURATION")
print("="*60)
print(f"K: {int(best['k'])}")
print(f"Silhouette: {best['silhouette']:.6f}")
print(f"Calinski-Harabasz: {best['calinski_harabasz']:.2f}")
print(f"Davies-Bouldin: {best['davies_bouldin']:.4f}")

if best['silhouette'] >= 0.4:
    print("\n🎉 ✅ TARGET ACHIEVED!")
else:
    gap = 0.4 - best['silhouette']
    print(f"\n⚠️  Gap to target: {gap:.4f}")

## 6. 🔬 Experiment 2: Different Clustering Algorithms

In [ ]:
# Use best K from previous experiment
best_k = int(results_df.loc[results_df['silhouette'].idxmax(), 'k'])

print("="*60)
print(f"TESTING DIFFERENT ALGORITHMS (K={best_k})")
print("="*60)

algorithm_results = []

# 1. K-Means (baseline)
print("\n1. K-Means: ", end='')
kmeans = KMeans(n_clusters=best_k, random_state=42)
labels_kmeans = kmeans.fit_predict(embeddings_norm)
sil_kmeans = silhouette_score(embeddings_norm, labels_kmeans, metric='cosine')
print(f"Silhouette = {sil_kmeans:.4f}")
algorithm_results.append({'algorithm': 'K-Means', 'silhouette': sil_kmeans, 'labels': labels_kmeans})

# 2. Gaussian Mixture Model
print("2. Gaussian Mixture: ", end='')
gmm = GaussianMixture(n_components=best_k, random_state=42, covariance_type='full')
labels_gmm = gmm.fit_predict(embeddings_norm)
sil_gmm = silhouette_score(embeddings_norm, labels_gmm, metric='cosine')
print(f"Silhouette = {sil_gmm:.4f}")
algorithm_results.append({'algorithm': 'Gaussian Mixture', 'silhouette': sil_gmm, 'labels': labels_gmm})

# 3. Spectral Clustering
print("3. Spectral Clustering: ", end='')
spectral = SpectralClustering(n_clusters=best_k, random_state=42, affinity='cosine')
labels_spectral = spectral.fit_predict(embeddings_norm)
sil_spectral = silhouette_score(embeddings_norm, labels_spectral, metric='cosine')
print(f"Silhouette = {sil_spectral:.4f}")
algorithm_results.append({'algorithm': 'Spectral', 'silhouette': sil_spectral, 'labels': labels_spectral})

# 4. Agglomerative (Hierarchical)
print("4. Agglomerative (Ward): ", end='')
agg = AgglomerativeClustering(n_clusters=best_k, linkage='ward')
labels_agg = agg.fit_predict(embeddings_norm)
sil_agg = silhouette_score(embeddings_norm, labels_agg, metric='cosine')
print(f"Silhouette = {sil_agg:.4f}")
algorithm_results.append({'algorithm': 'Agglomerative', 'silhouette': sil_agg, 'labels': labels_agg})

# 5. HDBSCAN (density-based, finds K automatically)
print("5. HDBSCAN (auto K): ", end='')
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, metric='cosine', min_samples=10)
labels_hdbscan = clusterer.fit_predict(embeddings_norm)

# Filter out noise points (-1 labels)
mask = labels_hdbscan >= 0
if mask.sum() > 0 and len(set(labels_hdbscan[mask])) > 1:
    sil_hdbscan = silhouette_score(embeddings_norm[mask], labels_hdbscan[mask], metric='cosine')
    n_clusters = len(set(labels_hdbscan[mask]))
    n_noise = (labels_hdbscan == -1).sum()
    print(f"Silhouette = {sil_hdbscan:.4f} (K={n_clusters}, noise={n_noise})")
    algorithm_results.append({'algorithm': f'HDBSCAN (K={n_clusters})', 'silhouette': sil_hdbscan, 'labels': labels_hdbscan})
else:
    print("Failed (all noise or single cluster)")

print("\n" + "="*60)

In [ ]:
# Plot algorithm comparison
alg_df = pd.DataFrame(algorithm_results)

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(alg_df['algorithm'], alg_df['silhouette'], color='skyblue', edgecolor='black')
ax.axvline(x=0.4, color='green', linestyle='--', linewidth=2, label='🎯 Target: 0.4')
ax.set_xlabel('Silhouette Score', fontsize=12)
ax.set_title('Algorithm Comparison (CodeBERT Embeddings)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='x', alpha=0.3)

# Annotate bars
for i, (bar, val) in enumerate(zip(bars, alg_df['silhouette'])):
    ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.4f}', 
            va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Print best
best_alg = alg_df.loc[alg_df['silhouette'].idxmax()]
print("\n" + "="*60)
print("BEST ALGORITHM")
print("="*60)
print(f"Algorithm: {best_alg['algorithm']}")
print(f"Silhouette: {best_alg['silhouette']:.6f}")

if best_alg['silhouette'] >= 0.4:
    print("\n🎉 ✅ TARGET ACHIEVED!")
else:
    print(f"\n⚠️  Gap to target: {0.4 - best_alg['silhouette']:.4f}")

## 7. 🔬 Experiment 3: Hierarchical 2-Level Clustering

**Strategy:**
1. Level 1: Split into Code vs Academic (K=2)
2. Level 2: Sub-cluster each group separately
   - Code cluster → K=5 or best K for code questions
   - Academic cluster → K=10 or best K for academic questions

In [ ]:
print("="*60)
print("HIERARCHICAL 2-LEVEL CLUSTERING")
print("="*60)

# Level 1: Binary split (Code vs Academic)
print("\nLevel 1: Splitting Code vs Academic...")
kmeans_l1 = KMeans(n_clusters=2, random_state=42, n_init=20)
labels_l1 = kmeans_l1.fit_predict(embeddings_norm)
sil_l1 = silhouette_score(embeddings_norm, labels_l1, metric='cosine')
print(f"  Silhouette (Level 1): {sil_l1:.4f}")

# Identify which cluster is code vs academic
cluster_0_code_pct = sum([1 for i, dt in enumerate(data_types) if labels_l1[i] == 0 and dt == 'code']) / sum(labels_l1 == 0)
cluster_1_code_pct = sum([1 for i, dt in enumerate(data_types) if labels_l1[i] == 1 and dt == 'code']) / sum(labels_l1 == 1)

code_cluster_id = 0 if cluster_0_code_pct > cluster_1_code_pct else 1
academic_cluster_id = 1 - code_cluster_id

print(f"  Cluster {code_cluster_id}: {cluster_0_code_pct*100 if code_cluster_id==0 else cluster_1_code_pct*100:.1f}% code (CODE cluster)")
print(f"  Cluster {academic_cluster_id}: {(1-cluster_1_code_pct)*100 if academic_cluster_id==1 else (1-cluster_0_code_pct)*100:.1f}% academic (ACADEMIC cluster)")

# Get indices for each cluster
code_indices = np.where(labels_l1 == code_cluster_id)[0]
academic_indices = np.where(labels_l1 == academic_cluster_id)[0]

print(f"\n  Code cluster size: {len(code_indices)}")
print(f"  Academic cluster size: {len(academic_indices)}")

In [ ]:
# Level 2: Sub-cluster each group
print("\nLevel 2: Sub-clustering each group...")

# Find optimal K for code cluster
print("\n  Testing K for CODE cluster...")
code_embeddings = embeddings_norm[code_indices]
best_k_code = 5
best_sil_code = 0

for k in [3, 4, 5, 6, 8]:
    kmeans_code = KMeans(n_clusters=k, random_state=42)
    labels_code = kmeans_code.fit_predict(code_embeddings)
    sil_code = silhouette_score(code_embeddings, labels_code, metric='cosine')
    print(f"    K={k}: Silhouette={sil_code:.4f}")
    if sil_code > best_sil_code:
        best_sil_code = sil_code
        best_k_code = k

print(f"  → Best K for code: {best_k_code} (silhouette={best_sil_code:.4f})")

# Find optimal K for academic cluster
print("\n  Testing K for ACADEMIC cluster...")
academic_embeddings = embeddings_norm[academic_indices]
best_k_academic = 10
best_sil_academic = 0

for k in [5, 8, 10, 12, 15]:
    kmeans_academic = KMeans(n_clusters=k, random_state=42)
    labels_academic = kmeans_academic.fit_predict(academic_embeddings)
    sil_academic = silhouette_score(academic_embeddings, labels_academic, metric='cosine')
    print(f"    K={k}: Silhouette={sil_academic:.4f}")
    if sil_academic > best_sil_academic:
        best_sil_academic = sil_academic
        best_k_academic = k

print(f"  → Best K for academic: {best_k_academic} (silhouette={best_sil_academic:.4f})")

In [ ]:
# Final hierarchical clustering with best K values
print("\n" + "="*60)
print("FINAL HIERARCHICAL CLUSTERING")
print("="*60)

# Cluster code
kmeans_code_final = KMeans(n_clusters=best_k_code, random_state=42)
labels_code_final = kmeans_code_final.fit_predict(code_embeddings)

# Cluster academic
kmeans_academic_final = KMeans(n_clusters=best_k_academic, random_state=42)
labels_academic_final = kmeans_academic_final.fit_predict(academic_embeddings)

# Combine labels (offset academic labels to avoid overlap)
hierarchical_labels = np.zeros(len(embeddings_norm), dtype=int)
hierarchical_labels[code_indices] = labels_code_final
hierarchical_labels[academic_indices] = labels_academic_final + best_k_code  # Offset

# Calculate overall silhouette
sil_hierarchical = silhouette_score(embeddings_norm, hierarchical_labels, metric='cosine')

print(f"\nLevel 1: K=2 (Code vs Academic)")
print(f"  Silhouette: {sil_l1:.4f}")
print(f"\nLevel 2:")
print(f"  Code cluster: K={best_k_code}, Silhouette={best_sil_code:.4f}")
print(f"  Academic cluster: K={best_k_academic}, Silhouette={best_sil_academic:.4f}")
print(f"\nTotal clusters: {best_k_code + best_k_academic}")
print(f"Overall Silhouette: {sil_hierarchical:.6f}")

if sil_hierarchical >= 0.4:
    print("\n🎉 ✅ TARGET ACHIEVED WITH HIERARCHICAL CLUSTERING!")
else:
    gap = 0.4 - sil_hierarchical
    print(f"\n⚠️  Gap to target: {gap:.4f}")
    print("\n💡 Weighted silhouette (by cluster size) might be higher!")
    weighted_sil = (len(code_indices) * best_sil_code + len(academic_indices) * best_sil_academic) / len(embeddings_norm)
    print(f"   Weighted average: {weighted_sil:.4f}")

## 8. 📊 Final Summary & Best Configuration

In [ ]:
print("="*80)
print("FINAL SUMMARY: CODEBERT CLUSTERING EXPERIMENTS")
print("="*80)

print("\n📊 EXPERIMENT RESULTS:")
print(f"\n1. K-Means with different K:")
best_kmeans = results_df.loc[results_df['silhouette'].idxmax()]
print(f"   Best: K={int(best_kmeans['k'])}, Silhouette={best_kmeans['silhouette']:.4f}")

print(f"\n2. Different algorithms (K={best_k}):")
for _, row in alg_df.iterrows():
    print(f"   {row['algorithm']}: {row['silhouette']:.4f}")

print(f"\n3. Hierarchical 2-level:")
print(f"   Overall: {sil_hierarchical:.4f}")
print(f"   Level 1 (K=2): {sil_l1:.4f}")
print(f"   Level 2 Code (K={best_k_code}): {best_sil_code:.4f}")
print(f"   Level 2 Academic (K={best_k_academic}): {best_sil_academic:.4f}")

# Find absolute best
all_results = [
    ('K-Means', best_kmeans['silhouette']),
    ('Hierarchical', sil_hierarchical),
] + [(row['algorithm'], row['silhouette']) for _, row in alg_df.iterrows()]

best_overall = max(all_results, key=lambda x: x[1])

print("\n" + "="*80)
print("🏆 BEST OVERALL CONFIGURATION")
print("="*80)
print(f"Method: {best_overall[0]}")
print(f"Silhouette: {best_overall[1]:.6f}")
print(f"Improvement over baseline (0.0353): {(best_overall[1] - 0.0353) / 0.0353 * 100:+.1f}%")

if best_overall[1] >= 0.4:
    print("\n🎉 🎉 🎉 TARGET ACHIEVED! 🎉 🎉 🎉")
else:
    gap = 0.4 - best_overall[1]
    print(f"\n⚠️  Gap to target 0.4: {gap:.4f}")
    print("\n💡 NEXT STEPS TO REACH 0.4+:")
    print("   1. Try more training data (currently ~3500 questions)")
    print("   2. Test other code-specific models (GraphCodeBERT, CodeT5, StarEncoder)")
    print("   3. Feature engineering: Combine CodeBERT + statistical features")
    print("   4. Data cleaning: Remove outliers/noise")
    print("   5. Try softer clustering (Fuzzy C-Means, Soft K-Means)")

print("="*80)

## 9. 💾 Export Results

In [ ]:
# Save results
results_df.to_csv('codebert_kmeans_results.csv', index=False)
alg_df[['algorithm', 'silhouette']].to_csv('codebert_algorithm_comparison.csv', index=False)

import json
final_config = {
    'best_method': best_overall[0],
    'best_silhouette': float(best_overall[1]),
    'target_achieved': bool(best_overall[1] >= 0.4),
    'improvement_pct': float((best_overall[1] - 0.0353) / 0.0353 * 100),
    'baseline': 0.0353,
    'experiments': {
        'kmeans_best_k': int(best_kmeans['k']),
        'kmeans_best_silhouette': float(best_kmeans['silhouette']),
        'hierarchical_overall': float(sil_hierarchical),
        'hierarchical_level1': float(sil_l1),
        'hierarchical_code_k': int(best_k_code),
        'hierarchical_academic_k': int(best_k_academic)
    }
}

with open('codebert_best_config.json', 'w') as f:
    json.dump(final_config, f, indent=2)

print("✅ Results saved:")
print("  - codebert_kmeans_results.csv")
print("  - codebert_algorithm_comparison.csv")
print("  - codebert_best_config.json")
print("\n📥 Download these files from Colab sidebar!")